<a href="https://colab.research.google.com/github/CAIKAYUBAKA/hds5210-2023/blob/main/midterm/midterm-2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """(str, int, int, float, int, int, float, str, str, str)-->float
    This function calculates the PRIEST Clinical Severity Score and return the risk percentage.
    Returns:
        float or None: Risk percentage between 0 and 1. Returns None for invalid inputs
        """
    # Initialize the score to 0
    score = 0
    points = 0

    # Calculate Sex score
    if sex.lower() == 'male':
        score += 1
    elif sex.lower() == 'female':
        score += 0
    else:
        return None

    # Calculate Age score
    if 16 <= age <= 49:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4
    else:
        return None
      # Calculate respiratory rate
    if respiratory_rate < 9:
        points += 3
    elif 9 <= respiratory_rate <= 11:
        points += 1
    elif 21 <= respiratory_rate <= 24:
        points += 2
    elif respiratory_rate > 24:
        points += 3
        # Calculate oxygen saturation
    if oxygen_saturation > 95:
        points += 0
    elif 94 <= oxygen_saturation <= 95:
        points += 1
    elif 92 <= oxygen_saturation <= 93:
        points += 2
    elif oxygen_saturation < 92:
        points += 3

    # Calculate heart rate
    if heart_rate < 41:
        points += 3
    elif 41 <= heart_rate <= 50:
        points += 1
    elif 51 <= heart_rate <= 90:
        points += 0
    elif 91 <= heart_rate <= 110:
        points += 1
    elif 111 <= heart_rate <= 130:
        points += 2
    elif heart_rate > 130:
        points += 3

    # Calculate systolic BP
    if systolic_bp < 91:
        points += 3
    elif 91 <= systolic_bp <= 100:
        points += 2
    elif 101 <= systolic_bp <= 110:
        points += 1
    elif 111 <= systolic_bp <= 219:
        points += 0
    elif systolic_bp > 219:
        points += 3

    # Calculate temperature
    if temperature < 35.1:
        points += 3
    elif 35.1 <= temperature <= 36.0:
        points += 1
    elif 36.1 <= temperature <= 38.0:
        points += 0
    elif 38.1 <= temperature <= 39.0:
        points += 1
    elif temperature > 39.0:
        points += 2

    # Calculate alertness
    if alertness.lower() == 'alert':
        points += 0
    elif alertness.lower() == 'confused or not alert':
        points += 3

    # Calculate inspired oxygen
    if inspired_oxygen.lower() == 'air':
        points += 0
    elif inspired_oxygen.lower() == 'supplemental oxygen':
        points += 2

    # Calculate performance status
    if performance_status.lower() == 'unrestricted normal activity':
        points += 0
    elif performance_status.lower() == 'limited strenuous activity, can do light activity':
        points += 1
    elif performance_status.lower() == 'limited activity, can self-care':
        points += 2
    elif performance_status.lower() == 'limited self-care':
        points += 3
    elif performance_status.lower() == 'bed/chair bound, no self-care':
        points += 4

  # Calculate risk percentage based on PRIEST score
    risk_percentage = {
        0: 0.01, 1: 0.01, 2: 0.02, 3: 0.02, 4: 0.03, 5: 0.09, 6: 0.15, 7: 0.18, 8: 0.22, 9: 0.26,
        10: 0.29, 11: 0.34, 12: 0.38, 13: 0.46, 14: 0.47, 15: 0.49, 16: 0.55, 17: 0.59, 18: 0.59,
        19: 0.59, 20: 0.59, 21: 0.59, 22: 0.59, 23: 0.59, 24: 0.59, 25: 0.59
    }.get(score, 0.99) if score >= 17 else 0.99

    return risk_percentage


In [2]:
risk_percentage_1 = priest('male', 30, 10, 0.96, 70, 120, 37.0, 'alert', 'air', 'unrestricted normal activity')  # Output: 0.59
risk_percentage_2 = priest('female', 70, 15, 0.91, 85, 140, 38.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')  # Output: 0.99
risk_percentage_3 = priest('unknown', 50, 20, 0.94, 60, 100, 36.8, 'confused or not alert', 'supplemental oxygen', 'limited activity, can self-care')  # Output: None

print(risk_percentage_1)
print(risk_percentage_2)
print(risk_percentage_3)


0.99
0.99
None


In [3]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [4]:
def find_hospital(age, sex, risk):
    """ (int, str, float)-->str
    This function will find the appropriate hospital based on age, sex, and risk percentage.
    Returns:
        str or None: JSON string containing age, sex, risk, and hospital name, or None if the request fails.
    """
    # Define the base URL of the REST web service
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=50&sex=male&risk_pct=0.2"

    # Define the URL with parameters
    url = f"{base_url}?age={age}&sex={sex.lower()}&risk_pct={risk}"

    try:
        # Make a GET request to the web services
        response = requests.get(url)

        if response.status_code == 200:
            # Parse the JSON response and extract the hospital name
            hospital_info = response.json()

            # My JSON dictionary should return as
            result_dict = {
                "age": str(age),
                "sex": sex,
                "risk": str(risk),
                "hospital": hospital_info.get("hospital")
            }

            # Convert my dictionary to a JSON string
            return json.dumps(result_dict)
        else:
            print(f"Error: Request failed with status code {response.status_code}")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None


In [11]:
result = find_hospital(50, 'male', 0.2)
print(f"{result}")


{"age": "50", "sex": "male", "risk": "0.2", "hospital": "Southwest Hospital and Medical Center"}


In [6]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [7]:
import requests
import json

def download_file(url, local_filename):
    response = requests.get(url)
    with open(local_filename, 'wb') as f:
        f.write(response.content)

url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
local_filename = "ga_hospitals.json"

download_file(url, local_filename)

def get_address(hospital_name):
    with open(local_filename, 'r') as f:
        data = json.load(f)
        hospital_info = data.get(hospital_name)
        if hospital_info:
            return hospital_info.get("ADDRESS")
        else:
            return None



In [12]:
address = get_address("Emory University Hospital Midtown")
print(address)


None


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [9]:
import requests

def calc_priest_score(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired):
    """(str, int, int, float, int, int, float, str, str, str)-->int
    This function calculates the PRIEST Clinical Severity Score and return the risk percentage.
    Returns:
        float or None: Risk percentage between 0 and 1. Returns None for invalid inputs.
"""

    pass

def process_people(file_location):
    """
   This function will process patient data and calculate PRIEST scores.
    Returns a list: A list of dictionaries where each dictionary contains patient information and calculated PRIEST score.

    """
    response = requests.get(file_location)
    content = response.content.decode('utf-8').split('\n')[1:-1]

    processed_data = []

    for line in content:
        items = line.split('|')
        patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = items


        # Calculate PRIEST score
        priest_score = calc_priest_score(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired)

        processed_data.append({
            'patient_number': patient_number,
            'sex': sex,
            'age': age,
            'breath': breath,
            'o2sat': o2sat,
            'heart': heart,
            'systolic': systolic,
            'temp': temp,
            'alertness': alertness,
            'inspired': inspired,
            'status': status,
            'priest_score': priest_score
        })

    return processed_data

file_location = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
processed_data = process_people(file_location)

# Printing the processed data for the first 5 patients
for data in processed_data[:5]:
    print(data)


{'patient_number': 'E9559', 'sex': 'FEMALE', 'age': '40', 'breath': '24', 'o2sat': '0.96', 'heart': '105', 'systolic': '115', 'temp': '34.9', 'alertness': 'ALERT', 'inspired': 'AIR', 'status': 'unrestricted normal activity', 'priest_score': None}
{'patient_number': 'E9385', 'sex': 'Female', 'age': '51', 'breath': '19', 'o2sat': '0.99', 'heart': '90', 'systolic': '91', 'temp': '38.1', 'alertness': 'ALERT', 'inspired': 'supplemental oxygen', 'status': 'unrestricted normal activity', 'priest_score': None}
{'patient_number': 'E3067', 'sex': 'female', 'age': '40', 'breath': '29', 'o2sat': '0.96', 'heart': '105', 'systolic': '95', 'temp': '38.1', 'alertness': 'ALERT', 'inspired': 'air', 'status': 'unrestricted normal activity', 'priest_score': None}
{'patient_number': 'E9422', 'sex': 'FEMALE', 'age': '66', 'breath': '19', 'o2sat': '0.96', 'heart': '135', 'systolic': '115', 'temp': '38.1', 'alertness': 'ALERT', 'inspired': 'AIR', 'status': 'bed/chair bound, no self-care', 'priest_score': None

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [10]:
import requests
import json

# Download the expected results JSON file
url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
response = requests.get(url)

if response.status_code == 200:
    # Save the content to a local file
    with open("people_results.json", "wb") as f:
        f.write(response.content)

    # Load the expected results from the local file
    with open("people_results.json", "r") as f:
        expected_results = json.load(f)

    # Now you can compare your results with the expected results
    for patient_data in processed_data:
        patient_number = patient_data['patient_number']
        expected_data = ((item for item in expected_results if item['patient_number'] == patient_number), None)

        if expected_data is None:
            print(f"Patient {patient_number} not found in expected results.")
            continue

        if patient_data != expected_data:
            print(f"Patient {patient_number} has different data:")
            print(f"Expected: {expected_data}")
            print(f"Actual: {patient_data}")

    print("Finished checking results.")
else:
    print(f"Error: Request failed with status code {response.status_code}")


Patient E9559 has different data:
Expected: (<generator object <genexpr> at 0x7941600f4580>, None)
Actual: {'patient_number': 'E9559', 'sex': 'FEMALE', 'age': '40', 'breath': '24', 'o2sat': '0.96', 'heart': '105', 'systolic': '115', 'temp': '34.9', 'alertness': 'ALERT', 'inspired': 'AIR', 'status': 'unrestricted normal activity', 'priest_score': None}
Patient E9385 has different data:
Expected: (<generator object <genexpr> at 0x7941600f44a0>, None)
Actual: {'patient_number': 'E9385', 'sex': 'Female', 'age': '51', 'breath': '19', 'o2sat': '0.99', 'heart': '90', 'systolic': '91', 'temp': '38.1', 'alertness': 'ALERT', 'inspired': 'supplemental oxygen', 'status': 'unrestricted normal activity', 'priest_score': None}
Patient E3067 has different data:
Expected: (<generator object <genexpr> at 0x7941600f4580>, None)
Actual: {'patient_number': 'E3067', 'sex': 'female', 'age': '40', 'breath': '29', 'o2sat': '0.96', 'heart': '105', 'systolic': '95', 'temp': '38.1', 'alertness': 'ALERT', 'inspire

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---